In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

### 1. Number of columns in the dataframe

In [3]:
df.shape[1]

19

### 2. Standard deviation for the duration feature

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
df['duration'] = pd.to_datetime(df.tpep_dropoff_datetime) - pd.to_datetime(df.tpep_pickup_datetime)
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

In [6]:
df.duration.std()

42.594351241920904

### 3. Dropping outliers

In [7]:
total = df.shape[0]
total

3066766

In [8]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [9]:
remaining = df.shape[0]

In [10]:
(remaining/total) * 100

98.1220282212598

### 4. One hot encoding

In [11]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')
train_dicts[:5]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

515

### 5. Training

In [13]:
y_train = df.duration

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = lr.predict(X_train)

In [16]:
mean_squared_error(y_pred, y_train, squared=False)

7.6492610279057605

### 6. Evaluation

In [17]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

In [18]:
df_val['duration'] = pd.to_datetime(df_val.tpep_dropoff_datetime) - pd.to_datetime(df_val.tpep_pickup_datetime)
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds()/60)

In [19]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [20]:
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')

In [21]:
X_val = dv.transform(val_dicts)

In [22]:
y_val = df_val.duration

In [23]:
y_pred = lr.predict(X_val)
mean_squared_error(y_pred, y_val, squared=False)

7.81183265470218